# Contract Types

Interactive Brokers supports various contract types. Each type has specific parameters:

- **Contract(conId)**: Direct reference using IB's contract ID
- **Stock**: Equity securities (symbol, exchange, currency, primaryExchange optional)
- **Forex**: Currency pairs (currency pair like 'EURUSD')
- **CFD**: Contracts for Difference (symbol)
- **Future**: Futures contracts (symbol, expiry date, exchange)
- **Option**: Options contracts (symbol, expiry, strike, right, exchange)
- **Bond**: Fixed income securities (using ISIN or other identifiers)

In [1]:
from ib_async import *
util.startLoop()

In [2]:
ib = IB()
ib.connect()

<IB connected to 127.0.0.1:7497 clientId=1>

## Contract Type Examples

In [3]:
# Contract(conId=270639)
# Stock('AMD', 'SMART', 'USD')
# Stock('INTC', 'SMART', 'USD', primaryExchange='NASDAQ')
# Forex('EURUSD')
# CFD('IBUS30')
# Future('ES', '20180921', 'GLOBEX')
# Option('SPY', '20170721', 240, 'C', 'SMART')
# Bond(secIdType='ISIN', secId='US03076KAA60')

## Handling Ambiguous vs Unambiguous Contracts

When creating contracts, some may be **ambiguous** (multiple matches) while others are **unambiguous** (single match):

- **Unambiguous**: When you provide enough details (symbol + exchange + currency), IB can uniquely identify the contract
- **Ambiguous**: When details are incomplete (e.g., just symbol), IB may find multiple matching contracts

The `qualifyContracts()` function behaves differently in each case:

In [4]:
contract = Stock("AAPL", "SMART", "USD")
contract

Stock(symbol='AAPL', exchange='SMART', currency='USD')

**Unambiguous case**: Full contract details provided (symbol, exchange, currency):

In [5]:
contract = ib.qualifyContracts(contract)  # unambiguous
contract

[Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS')]

This should return a single qualified contract since the details uniquely identify AAPL:

In [6]:
contract = ib.qualifyContracts(Stock("AAPL"))  # ambiguous
contract

Ambiguous contract: Stock(symbol='AAPL'), possibles are [Contract(secType='STK', conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), Contract(secType='STK', conId=273982664, symbol='AAPL', exchange='SMART', primaryExchange='EBS', currency='CHF', localSymbol='AAPL', tradingClass='AAPL'), Contract(secType='STK', conId=532640894, symbol='AAPL', exchange='SMART', primaryExchange='TSE', currency='CAD', localSymbol='AAPL', tradingClass='AAPL'), Contract(secType='STK', conId=265598, symbol='AAPL', exchange='AMEX', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), Contract(secType='STK', conId=265598, symbol='AAPL', exchange='NYSE', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), Contract(secType='STK', conId=265598, symbol='AAPL', exchange='CBOE', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), Contract(secType='ST

[None]

**Ambiguous case**: Only symbol provided, no exchange or currency specified. This may return multiple contracts if AAPL trades on different exchanges or in different currencies:

In [7]:
ib.disconnect()

'Disconnecting from 127.0.0.1:7497, 205 B sent in 10 messages, 48.9 kB received in 484 messages, session time 14.0 s.'